In [ ]:
%matplotlib inline

import numpy as np
import random
from scipy.stats import norm
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import QuantileRegressor
import matplotlib.pyplot as plt

seed = 0
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## Simple NN for XOR

In [ ]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

model = keras.Sequential()
model.add(keras.layers.Dense(units = 16, input_dim=2, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))

model.summary()


# State the loss function and optimizer (adam is a good choice usually)
model.compile(optimizer=keras.optimizers.Adam(),loss='mean_squared_error',metrics=['accuracy'])

losses = model.fit(X,y,epochs=500,verbose=2)

# View improvement over epochs
plt.plot(losses.history['loss'])
plt.show()
plt.plot(losses.history['accuracy']) # Notice we are able to plot the accuracy in training as well
plt.show()

In [ ]:
prob = model.predict(X) # Make predictions
pred = (prob > 0.5) + 0
#Evaluate the accuracy
print(np.mean(pred == y)) # Accuracy
# Since this is training data, compare with the final training accuracy
print(losses.history['accuracy'][-1])
print(confusion_matrix(y_pred=pred , y_true=y)) # Confusion matrix

In [ ]:
# Go back to cell 2 and consider other network structures
# What are the mathematical formulations for these structures?

## In case you want to save or load a model
#model.save('path/to/location')
#reload_model = keras.models.load_model('path/to/location')

## Quadratic example

In [ ]:
## Nonlinear relationship
N = 100
x = np.random.normal(loc=0,scale=1,size=N)
epsilon = np.random.normal(loc=0,scale=2,size=N)
B0 = 2
B2 = 3
y = B0 + B2*x**2 + epsilon

x_test = np.arange(start=min(x),stop=max(x),step=0.01)
y_true = B0 + B2*x_test**2

plt.scatter(x,y,color='k')
plt.plot(x_test,y_true,color='k',linewidth=2)

In [ ]:
# Run a linear regression
xx = np.vstack([np.ones(len(x)),x]).T
b0,b1 = np.linalg.lstsq(xx,y,rcond=None)[0]
print([b0,b1])

# Plot response
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true,color='k',linewidth=2)
plt.axline((0,b0),slope=b1,color='r',linewidth=2)

# Compute MSE
print(np.mean(((b0+b1*x_test) - y_true)**2))

In [ ]:
# Run a quadratic regression (since we know this is the ground truth)
xx = np.vstack([np.ones(len(x)),x,x**2]).T
b0,b1,b2 = np.linalg.lstsq(xx,y,rcond=None)[0]
print([b0,b1,b2])

# Predict on test data
y_quad = b0 + b1*x_test + b2*x_test**2

# Plot response
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true,color='k',linewidth=2)
plt.plot(x_test,y_quad,color='r',linewidth=2)

# Compute MSE
print(np.mean((y_quad - y_true)**2))

In [ ]:
# Consider a neural network
# What is the shape of this network?
HIDDEN = 16
NN =keras.Sequential()
NN.add(keras.layers.Dense(HIDDEN, input_dim=1,activation='relu')) #1 input x
NN.add(keras.layers.Dense(1, activation='linear')) #1 output y

NN.summary()

# State the loss function and optimizer (adam is a good choice usually)
NN.compile(optimizer=keras.optimizers.Adam(),loss='mean_squared_error')

# Fit the model to training data
EPOCHS = 500 # How long to train for
history = NN.fit(x,y,epochs=EPOCHS,verbose=0)
# View improvement over epochs
plt.plot(history.history['loss'])
plt.show()

# Make predictions on the test data
y_NN = NN.predict(x_test)

# Plot response
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true,color='k',linewidth=2)
plt.plot(x_test,y_quad,color='r',linewidth=2)
plt.plot(x_test,y_NN,color='b',linewidth=2)

# Compute MSE
print(np.mean((y_NN - y_true)**2))

In [ ]:
# How about a *deep* neural network
# What is the structure of this neural network?
# Consider a neural network
# What is the shape of this network?
HIDDEN = 16
NN =keras.Sequential()
NN.add(keras.layers.Dense(HIDDEN, input_dim=1,activation='relu')) #1 input x
NN.add(keras.layers.Dense(HIDDEN, activation='tanh'))
NN.add(keras.layers.Dense(1, activation='linear')) #1 output y

NN.summary()

# State the loss function and optimizer (adam is a good choice usually)
NN.compile(optimizer=keras.optimizers.Adam(),loss='mean_squared_error')

# Fit the model to training data
EPOCHS = 1000 # How long to train for (let this train longer)
history = NN.fit(x,y,epochs=EPOCHS,verbose=0)
# View improvement over epochs
plt.plot(history.history['loss'])
plt.show()

# Make predictions on the test data
y_NN = NN.predict(x_test)

# Plot response
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true,color='k',linewidth=2)
plt.plot(x_test,y_quad,color='r',linewidth=2)
plt.plot(x_test,y_NN,color='b',linewidth=2)

# Compute MSE
print(NN.evaluate(x_test, y_true, verbose=0)) # Another way to compute the MSE (losses)
# Let's go back and consider other network structures
# What are the mathematical formulations for these structures?

In [ ]:
print(NN.evaluate(x_test, y_true, verbose=0)) # Another way to compute the MSE (losses)

## Quantile regression (changing the loss function)

In [ ]:
# Consider the 10% quantile
# Test/plot fits
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true+norm.ppf(0.1,scale=2),color='r',linewidth=2)

# Run a quadratic quantile regression (since we know this is the ground truth)
quad_reg = QuantileRegressor(quantile=0.1, alpha=0) #alpha is for a Lasso-type penalty term
X = np.transpose(np.vstack((x,x**2)))
quad_reg.fit(X, y)
print((quad_reg.intercept_ , quad_reg.coef_))

X_test = np.transpose(np.vstack((x_test,x_test**2)))
y_quad_pred = quad_reg.predict(X_test)
plt.plot(x_test,y_quad_pred,color='b',linewidth=2)

In [ ]:
nn_reg = keras.Sequential()
nn_reg.add(keras.layers.Dense(16, input_dim=1,activation='relu')) #1 input x
nn_reg.add(keras.layers.Dense(16, activation='relu'))
nn_reg.add(keras.layers.Dense(1, activation='linear')) #1 output y

nn_reg.summary()

# We need to create our own loss function
import keras.backend as K
def quantile_loss(q):
    def ql(y_true , y_pred): 
        e = y_true - y_pred
        #if e > 0 then qe > (q-1)e; if e < 0 then qe < (q-1)e
        loss = K.mean(K.maximum(q*e , (q-1)*e)) 
        return loss
    
    return ql

# State the loss function and optimizer (adam is a good choice usually)
nn_reg.compile(optimizer=keras.optimizers.Adam(),loss=quantile_loss(0.1)) #Now we minimize our custom loss function

# Fit the model to training data
losses = nn_reg.fit(x,y,epochs=500,verbose=0) # Fit the model to training data
# View improvement over epochs
plt.plot(losses.history['loss'])
plt.show()

In [ ]:
# Predict with the neural network
y_nn_pred = nn_reg.predict(x_test)
plt.scatter(x,y,color='k')
plt.plot(x_test,y_true+norm.ppf(0.1,scale=2),color='r',linewidth=2)
plt.plot(x_test,y_quad_pred,color='b',linewidth=2)
plt.plot(x_test,y_nn_pred,color='g',linewidth=2)

## Feedforward NN for financial data

In [ ]:
# Download financial data:
import yfinance
from datetime import datetime

#myData = DataReader(["IBM"],"yahoo",datetime(2010,1,1),datetime(2021,12,31)) #IBM chosen at random
#IBM = myData["Adj Close"]["IBM"]
myData = yfinance.download(["IBM"],datetime(2010,1,1),datetime(2021,12,31))
IBM = myData["Adj Close"]

r = np.log(IBM) - np.log(IBM.shift(1)) # Daily log return
r = r.to_numpy()
r = np.delete(r , 0) # Remove first date because 1 lag in returns

train_X = r[0:2500]
test_X = r[2500:3018]
train_y = r[1:2501]
test_y = r[2501:3019]

# Construct neural network
HIDDEN = 16
NN =keras.Sequential()
NN.add(keras.layers.Dense(HIDDEN, input_dim=1,activation='relu')) #1 input x
NN.add(keras.layers.Dense(HIDDEN, activation='relu'))
NN.add(keras.layers.Dense(1, activation='linear')) #1 output y

NN.summary()

# State the loss function and optimizer (adam is a good choice usually)
NN.compile(optimizer=keras.optimizers.Adam(),loss='mean_squared_error')

# Fit the model to training data
EPOCHS = 100 # How long to train for (cut off early)
history = NN.fit(train_X,train_y,epochs=EPOCHS,validation_split=0.1,verbose=0)
# View improvement over epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Training','Validation'])
plt.show()
print(history.history['loss'][-1]) # Training MSE
print(history.history['val_loss'][-1]) # Validation loss

# Evaluate on test data
print(NN.evaluate(test_X , test_y , verbose=0)) # Test MSE

In [ ]:
# How does this compare with an AR(1) model
XX = np.vstack([np.ones(len(train_X)),train_X]).T
b0,b1 = np.linalg.lstsq(XX,train_y,rcond=None)[0]

# Compute MSE
print(np.mean(((b0+b1*test_X) - test_y)**2))


# How should we modify this NN to consider 2+ lags?
# What other network structures should we try?